# Costumer segmentation - k-prototypes

tipos de segmentação de consumidores

- comportamental: padrões de consumo, quantidade e qualidade
- psicográfica: estilo de vida, opiniões, crenças
- geográfica: estado, cidade, bairro
- demográfica: gênero, estado civil, idade, profissão, etc
- value-based: a receita que eles geram

passos do projeto

- entender e preparar os dados
- encontrar o número de clusters usando o algoritmo k-prototypes
- plotar os clusters

### 0. Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

### 1. Base de dados

In [4]:
data = pd.read_csv('Data.csv')

# deletar colunas desnecessárias
data.drop(['ID', 'Unnamed: 0'], axis = 'columns', inplace = True)

In [5]:
data.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4
1,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4
2,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6
3,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6
4,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6


In [7]:
data.shape

(8068, 9)

In [10]:
data.isnull().sum()

Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
dtype: int64

In [11]:
data.describe()

,Age,Work_Experience,Family_Size
count,8068.000000,7239.000000,7733.000000
mean,43.466906,2.641663,2.850123
std,16.711696,3.406763,1.531413
min,18.000000,0.000000,1.000000
25%,30.000000,0.000000,2.000000
50%,40.000000,1.000000,3.000000
75%,53.000000,4.000000,4.000000
max,89.000000,14.000000,9.000000


In [12]:
data.columns

Index(['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1'],
      dtype='object')

In [14]:
# quais são as categorias nas variáveis categóricas?

df_cat = data[['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']]

for i in df_cat.columns:
    print(i, df_cat[i].unique())

Gender ['Male' 'Female']
Ever_Married ['No' 'Yes' nan]
Graduated ['No' 'Yes' nan]
Profession ['Healthcare' 'Engineer' 'Lawyer' 'Entertainment' 'Artist' 'Executive'
 'Doctor' 'Homemaker' 'Marketing' nan]
Spending_Score ['Low' 'Average' 'High']
Var_1 ['Cat_4' 'Cat_6' 'Cat_7' 'Cat_3' 'Cat_1' 'Cat_2' nan 'Cat_5']
